In [1]:
import requests
from bs4 import BeautifulSoup
import re

__1968 - 1981__

In [2]:
import pandas as pd


In [ ]:
page = requests.get("https://www.dol.gov/agencies/whd/state/minimum-wage/history")
soup = BeautifulSoup(page.content, 'html.parser')
tabelas = soup.find_all('table')

df_wage = []
for i, tabela in enumerate(tabelas):
    linhas = tabela.find_all('tr')
    cabecalho = linhas[0]
    anos = [th.text for th in cabecalho.find_all('th')[1:]]
    estados = []
    for estado in linhas[1:]:
        estados.append([td.text for td in estado.find_all('td')])
    df = pd.DataFrame(estados, columns=['state'] + anos)
    df_wage.append(df)
df = pd.concat(df_wage, ignore_index=True)

footnotes = soup.find('div', id='content')
footnotes.find_all('p')
list_footnotes = []
for p in footnotes.find_all('p'):
    if re.match(r'^[\[\(].[\]\)]', p.text):
        id_footnote = p.text.strip().split(' ')[0]
        text_footnote = ' '.join(p.text.strip().split(' ')[1:]).replace('- ','')
        list_footnotes.append((id_footnote, text_footnote))
footnotes_dict = {id_: text for id_, text in list_footnotes}
columns_to_adjust = [col for col in df.columns if  not col.isnumeric() and col != 'Estado']
footnote_year_bridge = {}
for key, _ in footnotes_dict.items():
    for col in columns_to_adjust:
        if key in col:
            footnote_year_bridge[col.replace(key, '').strip()] = key
            df = df.rename(columns={col: col.replace(key, '').strip()})

df_melted = df.melt(id_vars=['state'], var_name='year', value_name='minimal_wage').dropna()
df_melted['year'] = df_melted['year'].astype(int)
df_melted['minimal_wage'] = df_melted['minimal_wage'].str.replace('$', '')
df_melted['id'] = df_melted.index + 1

df_melted['minimal_wage'] = df_melted['minimal_wage'].str.replace(r'[\[\(].*?[\]\)]', '', regex=True)
df_melted['minimal_wage'] = df_melted['minimal_wage'].mask(
    df_melted['minimal_wage'].isin(['...', 'NA']), 
    pd.NA
)
if 'notes' not in df_melted.columns:
    df_melted['notes'] = pd.NA

if 'frequency' not in df_melted.columns:
    df_melted['frequency'] = pd.NA


def add_leading_zero(value):
    value = value.strip()
    if value.startswith('.'):
        return '0' + value
    return value

# Função para processar valores com múltiplas taxas
def process_multiple_rates(row):
    wage = row['minimal_wage']
    
    if pd.notna(wage) and isinstance(wage, str):
        original_wage = wage
        
        # 1. Detectar e remover frequency markers
        frequency = None
        if '/day' in wage:
            frequency = 2
            wage = wage.replace('/day', '').strip()
        elif '/wk' in wage:
            frequency = 3
            wage = wage.replace('/wk', '').strip()
        
        # 2. Detectar múltiplos valores com regex mais robusto
        # Padrão: captura valores monetários separados por -, &, /, ou espaços
        pattern = r'\$?\d+\.?\d*'
        matches = re.findall(pattern, wage)
        
        if len(matches) >= 2:
            first_value = add_leading_zero(matches[0])
            second_value = add_leading_zero(matches[1])
            
            row['minimal_wage'] = first_value
            note = f"Or can be {second_value}, this reflects which rates differ by industry, occupation or other factors, as established under a wage-board type law"
            row['notes'] = note
        elif len(matches) == 1:
            row['minimal_wage'] = add_leading_zero(matches[0])
        else:
            row['minimal_wage'] = add_leading_zero(wage)
        
        # 3. Atualizar frequency
        if frequency is not None:
            row['frequency'] = frequency
    
    # Garantir valor padrão para frequency
    if pd.isna(row['frequency']):
        row['frequency'] = 1
    
    return row
# Aplicar a função
df_melted = df_melted.apply(process_multiple_rates, axis=1)
df_melted['minimal_wage'] = df_melted['minimal_wage'].astype(str).str.extract(r'([\d.]+)', expand=False)

def notes_for_null_wage(row):
    if pd.isna(row['minimal_wage']) and pd.isna(row['notes']):
        return "This state utilizes the federal minimum wage"
    return row['notes']

df_melted['notes'] = df_melted.apply(notes_for_null_wage, axis=1)  
df_melted['minimal_wage'] = pd.to_numeric(df_melted['minimal_wage'], errors='coerce')
df_final = df_melted[['id', 'state', 'year', 'minimal_wage', 'frequency','notes']]


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def extrair_tabela_tipped_minimum_wage(year):
    url = f'https://www.dol.gov/agencies/whd/state/minimum-wage/tipped/{year}'
    tip_test = requests.get(url)
    if tip_test.status_code != 200:
        print(f"❌ Falha ao obter dados de {year} (status {tip_test.status_code})")
        return pd.DataFrame()
    
    tip_soup = BeautifulSoup(tip_test.content, 'html.parser')

    # 1. EXTRAIR FOOTNOTES
    footnotes_dict = {}
    for a_tag in tip_soup.find_all('a', attrs={'name': lambda x: x and x.startswith('foot')}):
        name = a_tag.get('name')
        parent_p = a_tag.find_parent('p')
        if parent_p:
            footnote_num = a_tag.get_text(strip=True)
            texto_completo = ' '.join(parent_p.get_text().split())
            texto_nota = texto_completo.replace(footnote_num, '', 1).strip()
            footnotes_dict[name] = texto_nota

    # 2. PROCESSAR TABELA
    tip_table = tip_soup.find('table')
    if not tip_table:
        print(f"⚠️ Nenhuma tabela encontrada em {year}")
        return pd.DataFrame()

    tip_linhas = tip_table.find_all('tr')[1:]
    header_order = ['jurisdiction', 'combinedrate', 'tipcredit', 'cashwage', 'definition']

    def processar_celula_valor(td_element, column_name):
        if not td_element:
            return None, None, []
        footnote_refs = []
        for link in td_element.find_all('a', href=True):
            href = link.get('href')
            if href:
                match = re.search(r'#(foot\d+)', href)
                if match:
                    footnote_refs.append(match.group(1))
        td_html = str(td_element)
        soup_copy = BeautifulSoup(td_html, 'html.parser')
        for link in soup_copy.find_all('a'):
            link.decompose()
        valor = ' '.join(soup_copy.get_text().split())
        footnote_texts = []
        for ref in footnote_refs:
            if ref in footnotes_dict:
                footnote_texts.append(f"[{column_name}] {footnotes_dict[ref]}")
        footnote_text = ' ; '.join(footnote_texts) if footnote_texts else None
        return valor if valor else None, footnote_text, footnote_refs

    def processar_jurisdiction(td_element):
        if not td_element:
            return None, None, None
        footnote_refs = []
        for link in td_element.find_all('a', href=True):
            href = link.get('href')
            if href:
                match = re.search(r'#(foot\d+)', href)
                if match:
                    footnote_refs.append(match.group(1))
        td_html = str(td_element)
        soup_copy = BeautifulSoup(td_html, 'html.parser')
        for link in soup_copy.find_all('a'):
            link.decompose()
        strong_tag = soup_copy.find('strong')
        if strong_tag:
            texto = ' '.join(strong_tag.get_text().split())
            nome_limpo = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
        else:
            nome_limpo = soup_copy.get_text(strip=True)
        extra_text = soup_copy.get_text().replace(nome_limpo, '').strip()
        footnote_texts = [footnotes_dict[ref] for ref in footnote_refs if ref in footnotes_dict]
        footnote_text = ' ; '.join(footnote_texts) if footnote_texts else None
        return nome_limpo, footnote_text, extra_text

    dados_tabela = []
    ultima_jurisdiction = None
    ultima_footnote = None
    for tr in tip_linhas:
        row_data = {}
        tds = tr.find_all('td')
        if tds and tds[0].get('colspan'):
            continue
        td_jurisdiction = tr.find('td', headers='jurisdiction')
        todas_notas = []
        if td_jurisdiction and td_jurisdiction.find('strong'):
            jurisdiction_limpa, footnote_text, extra_text = processar_jurisdiction(td_jurisdiction)
            ultima_jurisdiction = jurisdiction_limpa
            ultima_footnote = footnote_text
            row_data['jurisdiction'] = jurisdiction_limpa
            if footnote_text:
                todas_notas.append(footnote_text)
            if extra_text:
                todas_notas.append(extra_text)
        else:
            if ultima_jurisdiction:
                row_data['jurisdiction'] = ultima_jurisdiction
                if ultima_footnote:
                    todas_notas.append(ultima_footnote)
        for td in tds:
            header_name = td.get('headers')[0] if td.get('headers') else None
            if not header_name:
                header_name = header_order[tds.index(td)] if len(tds) == 4 else header_order[tds.index(td) - 1]
            valor_limpo, footnote_text, _ = processar_celula_valor(td, header_name)
            if header_name != 'jurisdiction':
                row_data[header_name] = valor_limpo
            if footnote_text:
                todas_notas.append(footnote_text)
        if todas_notas:
            row_data['notes'] = ' ; '.join(todas_notas)
        if row_data and any(v for k, v in row_data.items() if k not in ['jurisdiction', 'notes']):
            row_data['year'] = year
            dados_tabela.append(row_data)

    df_tips = pd.DataFrame(dados_tabela)
    return df_tips


# === LOOP PELOS ANOS 2003–2024 ===
dfs = []
for year in range(2024, 2025):
    df_year = extrair_tabela_tipped_minimum_wage(year)
    if not df_year.empty:
        dfs.append(df_year)

# Concatenar todos os DataFrames
df_tips = pd.concat(dfs, ignore_index=True)

# Mostrar resultado final
print(f"\n✅ Total de registros extraídos: {len(df_tips)}")

def process_tip_wages(row):
    """
    Processa valores de salário tipped, lidando com:
    - Múltiplos valores separados (-, &, /, espaços)
    - Textos descritivos movidos para notes
    - Porcentagens e valores especiais
    """
    
    def is_monetary_value(value):
        """Verifica se é um valor monetário válido"""
        if pd.isna(value) or not isinstance(value, str):
            return False
        # Remove espaços e verifica se tem formato de dinheiro
        clean = value.strip()
        # Padrão: pode ter $ e números com ponto decimal
        return bool(re.match(r'^\$?\d+\.?\d*$', clean))
    
    def is_percentage(value):
        """Verifica se é uma porcentagem"""
        if pd.isna(value) or not isinstance(value, str):
            return False
        return '%' in value or value.lower() in ['50%', 'to 50%']
    
    def extract_multiple_values(value):
        """Extrai múltiplos valores monetários de uma string"""
        if pd.isna(value) or not isinstance(value, str):
            return None
        
        # Procurar por múltiplos valores monetários
        pattern = r'\$?\d+\.?\d*'
        matches = re.findall(pattern, value)
        
        # Filtrar apenas valores que parecem dinheiro (com ou sem $)
        valid_matches = [m for m in matches if re.match(r'^\$?\d+\.\d+$', m)]
        
        return valid_matches if len(valid_matches) > 1 else None
    
    def move_text_to_notes(column_name, value, row):
        """Move texto descritivo para notes"""
        if pd.isna(value) or not isinstance(value, str):
            return value, row
        
        # Se não é valor monetário nem porcentagem, é texto descritivo
        if not is_monetary_value(value) and not is_percentage(value):
            # Adicionar à nota
            note_text = f"[{column_name}] {value}"
            
            if pd.notna(row.get('notes')) and row['notes'] != 'Missing value':
                row['notes'] = f"{row['notes']} ; {note_text}"
            else:
                row['notes'] = note_text
            
            return None, row  # Limpar o valor original
        
        return value, row
    
    # Processar cada coluna de valor
    for col in ['combinedrate', 'tipcredit', 'cashwage']:
        if col not in row:
            continue
            
        value = row[col]
        
        if pd.isna(value) or value == 'Missing value':
            continue
        
        # 1. Verificar se tem múltiplos valores
        multiple_values = extract_multiple_values(value)
        
        if multiple_values:
            # Tem múltiplos valores - usar o primeiro e criar nota
            first_value = multiple_values[0]
            if not first_value.startswith('$'):
                first_value = f'${first_value}'
            
            row[col] = first_value
            
            # Criar nota com os valores alternativos
            other_values = ', '.join(multiple_values[1:])
            note_text = f"[{col}] Alternative rate(s): {other_values}"
            
            if pd.notna(row.get('notes')) and row['notes'] != 'Missing value':
                row['notes'] = f"{row['notes']} ; {note_text}"
            else:
                row['notes'] = note_text
        
        # 2. Se não é valor monetário nem porcentagem, mover para notes
        else:
            value, row = move_text_to_notes(col, value, row)
            row[col] = value
    
    return row

# Aplicar a função
df_tips = df_tips.apply(process_tip_wages, axis=1)
df_tips[['combinedrate', 'tipcredit', 'cashwage']] = df_tips[['combinedrate', 'tipcredit', 'cashwage']].apply(lambda x: x.str.replace('$', '', regex=False))

def convert_with_context(value, column_name, row):
    """Converte e adiciona tipo na coluna + nota quando necessário"""
    if pd.isna(value):
        return None, None, row
    
    if not isinstance(value, str):
        return float(value) if isinstance(value, (int, float)) else None, 'exact', row
    
    original = value.strip()
    value = original.replace('$', '')
    
    if value.lower() in ['not specified', 'missing value', '']:
        return None, None, row
    
    # Porcentagem
    if '%' in value:
        match = re.search(r'(\d+\.?\d*)\s*%', value)
        if match:
            note = f"[{column_name}] Original value: {original}"
            if pd.notna(row.get('notes')) and row['notes'] != 'Missing value':
                row['notes'] = f"{row['notes']} ; {note}"
            else:
                row['notes'] = note
            return float(match.group(1)), 'percentage', row
    
    # Range
    range_patterns = {
        'up to': r'up to\s+(\d+\.?\d*)',
        'more than': r'more than\s+(\d+\.?\d*)',
        'at least': r'at least\s+(\d+\.?\d*)'
    }
    
    for range_type, pattern in range_patterns.items():
        match = re.search(pattern, value, re.IGNORECASE)
        if match:
            note = f"[{column_name}] {range_type.capitalize()} {match.group(1)}"
            if pd.notna(row.get('notes')) and row['notes'] != 'Missing value':
                row['notes'] = f"{row['notes']} ; {note}"
            else:
                row['notes'] = note
            return float(match.group(1)), 'range', row
    
    # Exato
    try:
        return float(value), 'exact', row
    except ValueError:
        return None, None, row

# Aplicar
def process_with_types(row):
    for col in ['combinedrate', 'tipcredit', 'cashwage']:
        if col in row:
            value, value_type, row = convert_with_context(row[col], col, row)
            row[col] = value
            row[f'{col}_type'] = value_type
    return row

df_tips = df_tips.apply(process_with_types, axis=1)
df_tips


✅ Total de registros extraídos: 54


,jurisdiction,combinedrate,tipcredit,cashwage,definition,notes,year,combinedrate_type,tipcredit_type,cashwage_type
0,FEDERAL,7.25,5.12,2.13,More than $30,: Fair Labor Standards Act (FLSA),2024,exact,exact,exact
1,Minnesota,NaN,NaN,NaN,Large employer: annual gross revenue of at lea...,Minnesota. A large employer means an enterpris...,2024,None,None,None
2,Minnesota,NaN,NaN,NaN,Small employer: annual gross revenue of less t...,Minnesota. A large employer means an enterpris...,2024,None,None,None
3,Montana,NaN,NaN,NaN,"Business with gross annual sales over $110,000",NaN,2024,None,None,None
4,Montana,NaN,NaN,NaN,Business not covered by the Fair Labor Standar...,NaN,2024,None,None,None
5,Arizona,14.35,3.00,11.35,Not specified,NaN,2024,exact,exact,exact
6,Arkansas,11.00,8.37,2.63,Not specified,NaN,2024,exact,exact,exact
7,Colorado,3.02,NaN,11.40,More than $30,NaN,2024,exact,None,exact
8,Connecticut,NaN,9.31,NaN,"Hotel, restaurant",Connecticut. The Connecticut minimum wage is a...,2024,None,exact,None
9,Connecticut,NaN,7.46,NaN,Bartenders who customarily receive tips,Connecticut. The Connecticut minimum wage is a...,2024,None,exact,None


In [4]:
import pandas as pd
df_test = pd.read_csv('output/fact_20251019_140838.csv')
df_test

,wage_id,state_id,category_id,year,effective_date,base_wage_per_hour,minimum_cash_wage,maximum_tip_credit,frequency,source_url,notes
0,1,1,1,1968,NaN,1.15,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,"Or can be 1.60, this reflects rates that diffe..."
1,2,2,1,1968,NaN,NaN,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,This state utilizes the federal minimum wage
2,3,3,1,1968,NaN,2.10,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,NaN
3,4,4,1,1968,NaN,18.72,NaN,NaN,3,https://www.dol.gov/agencies/whd/state/minimum...,"Or can be 26.40, this reflects rates that diff..."
4,5,5,1,1968,NaN,1.25,NaN,NaN,2,https://www.dol.gov/agencies/whd/state/minimum...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5245,5246,47,3,2024,NaN,NaN,NaN,9.87,1,https://www.dol.gov/agencies/whd/state/minimum...,NaN
5246,5247,47,4,2024,NaN,NaN,2.13,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,NaN
5247,5248,51,2,2024,NaN,7.25,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,NaN
5248,5249,51,3,2024,NaN,NaN,NaN,5.12,1,https://www.dol.gov/agencies/whd/state/minimum...,NaN


In [ ]:
df_tips

,jurisdiction,combinedrate,tipcredit,cashwage,definition,notes,year,combinedrate_type,tipcredit_type,cashwage_type
0,FEDERAL,5.15,3.02,2.13,More than $30,:\n \n Fair \n Labor Standa...,2003,exact,exact,exact
1,Alaska,NaN,NaN,7.15,NaN,"[cashwage] Alaska . Beginning January 1, 2004,...",2003,None,None,exact
2,California,NaN,NaN,6.75,NaN,NaN,2003,None,None,exact
3,Guam,NaN,NaN,5.15,NaN,NaN,2003,None,None,exact
4,Minnesota,NaN,NaN,5.15,Large employer,[definition] Minnesota . A large employer is a...,2003,None,None,exact
...,...,...,...,...,...,...,...,...,...,...
1384,Tennessee,7.25,5.12,2.13,More than $30,The following states do not have state minimum...,2024,exact,exact,exact
1385,Texas,7.25,5.12,2.13,More than $20,"The minimum wage laws in Kansas, Oklahoma (see...",2024,exact,exact,exact
1386,Utah,7.25,5.12,2.13,More than $30,"The minimum wage laws in Kansas, Oklahoma (see...",2024,exact,exact,exact
1387,Virginia,12.00,9.87,2.13,More than $30,NaN,2024,exact,exact,exact


[{'footnote_id': 'foot1',
  'footnote_number': '1',
  'footnote_text': 'Other additional deductions are permitted, for example for meals and lodging, except as noted in footnote 8 .'},
 {'footnote_id': 'foot2',
  'footnote_number': '2',
  'footnote_text': 'Alaska . Beginning January 1, 2004, and annually thereafter, the rate will be adjusted for inflation using either the Consumer Price Index for all urban consumers for Anchorage, Alaska, or $1 more than the Federal minimum wage, whichever is greater.'},
 {'footnote_id': 'foot3',
  'footnote_number': '3',
  'footnote_text': 'Minnesota . A large employer is an enterprise with annual receipts of $500,000 or more; a small employer, less than $500,000.'},
 {'footnote_id': 'foot4',
  'footnote_number': '4',
  'footnote_text': 'Oregon . Beginning January 1, 2004, and annually thereafter, the rate will be adjusted for inflation by a calculation using the U.S. City Average Consumer Price Index for All Urban Consumers for All Items. The wage am

In [ ]:
import pandas as pd
import hashlib
import re

# ============================================
# 1. PREPARAR DATASETS
# ============================================

# Assumindo que você já tem:
# - df_melted (salário mínimo normal)
# - df_tips (tipped wages processado)

# Padronizar colunas do df_melted
df_standard = df_melted.copy()
df_standard['category_name'] = 'Standard Minimum Wage'
df_standard['category_type'] = 'standard'
df_standard = df_standard.rename(columns={
    'state': 'jurisdiction',
    'minimal_wage': 'base_wage_per_hour'
})
df_standard['minimum_cash_wage'] = None
df_standard['maximum_tip_credit'] = None
df_standard['source_url'] = 'https://www.dol.gov/agencies/whd/state/minimum-wage/history'

# Transformar df_tips em formato long (uma linha por métrica)
df_tipped_long = []

for _, row in df_tips.iterrows():
    base_row = {
        'jurisdiction': row['jurisdiction'],
        'year': row['year'],
        'notes': row.get('notes'),
        'source_url': f"https://www.dol.gov/agencies/whd/state/minimum-wage/tipped/{row['year']}"
    }
    
    # Combined Rate
    if pd.notna(row.get('combinedrate')):
        df_tipped_long.append({
            **base_row,
            'category_name': 'Tipped Combined Rate',
            'category_type': 'tipped',
            'base_wage_per_hour': row['combinedrate'],
            'value_type': row.get('combinedrate_type'),
            'minimum_cash_wage': None,
            'maximum_tip_credit': None,
            'frequency': 1
        })
    
    # Tip Credit
    if pd.notna(row.get('tipcredit')):
        df_tipped_long.append({
            **base_row,
            'category_name': 'Tipped Credit',
            'category_type': 'tipped',
            'base_wage_per_hour': None,
            'value_type': row.get('tipcredit_type'),
            'minimum_cash_wage': None,
            'maximum_tip_credit': row['tipcredit'],
            'frequency': 1
        })
    
    # Cash Wage
    if pd.notna(row.get('cashwage')):
        df_tipped_long.append({
            **base_row,
            'category_name': 'Tipped Cash Wage',
            'category_type': 'tipped',
            'base_wage_per_hour': None,
            'value_type': row.get('cashwage_type'),
            'minimum_cash_wage': row['cashwage'],
            'maximum_tip_credit': None,
            'frequency': 1
        })

df_tipped_transformed = pd.DataFrame(df_tipped_long)

# ============================================
# 2. UNIFICAR DATASETS
# ============================================

# Garantir colunas comuns
common_columns = ['jurisdiction', 'year', 'category_name', 'category_type', 
                  'base_wage_per_hour', 'minimum_cash_wage', 'maximum_tip_credit',
                  'frequency', 'notes', 'source_url']

# Adicionar colunas faltantes
for col in common_columns:
    if col not in df_standard.columns:
        df_standard[col] = None
    if col not in df_tipped_transformed.columns:
        df_tipped_transformed[col] = None

# Concatenar
df_unified = pd.concat([
    df_standard[common_columns],
    df_tipped_transformed[common_columns]
], ignore_index=True)

# ============================================
# 3. CRIAR DIM_CATEGORIES
# ============================================

dim_categories = df_unified[['category_name', 'category_type']].drop_duplicates().reset_index(drop=True)
dim_categories['category_id'] = dim_categories.index + 1

# Merge category_id de volta
df_unified = df_unified.merge(
    dim_categories[['category_id', 'category_name']], 
    on='category_name', 
    how='left'
)

# ============================================
# 4. CRIAR DIM_STATES (se ainda não existir)
# ============================================

dim_states = df_unified[['jurisdiction']].drop_duplicates().reset_index(drop=True)
dim_states = dim_states.rename(columns={'jurisdiction': 'state_name'})
dim_states['state_id'] = dim_states.index + 1
dim_states['is_territory'] = False  # Ajustar conforme necessário

# Merge state_id
df_unified = df_unified.merge(
    dim_states.rename(columns={'state_name': 'jurisdiction'}),
    on='jurisdiction',
    how='left'
)

# ============================================
# 5. EXTRAIR E NORMALIZAR FOOTNOTES
# ============================================

def extract_footnotes_from_notes(notes_text):
    """
    Extrai footnotes individuais de uma string de notas
    Retorna: lista de (column_reference, footnote_text)
    """
    if pd.isna(notes_text) or not isinstance(notes_text, str):
        return []
    
    # Padrão: [column_name] texto ; [column_name] texto
    pattern = r'\[([^\]]+)\]\s*([^;]+)'
    matches = re.findall(pattern, notes_text)
    
    footnotes = []
    for column_ref, text in matches:
        text_clean = text.strip()
        if text_clean:
            footnotes.append((column_ref.strip(), text_clean))
    
    # Se não tem padrão [column], considerar nota geral
    if not matches and notes_text.strip():
        footnotes.append(('general', notes_text.strip()))
    
    return footnotes

def generate_footnote_hash(text):
    """Gera hash único para identificar footnotes duplicados"""
    return hashlib.md5(text.encode('utf-8')).hexdigest()[:16]

# Coletar todos os footnotes únicos
all_footnotes = {}  # {hash: (text, first_occurrence_id)}
bridge_data = []    # Lista de (wage_id, footnote_hash, column_reference)

for idx, row in df_unified.iterrows():
    notes = row.get('notes')
    if pd.notna(notes):
        footnotes = extract_footnotes_from_notes(notes)
        
        for column_ref, text in footnotes:
            fn_hash = generate_footnote_hash(text)
            
            # Adicionar ao dicionário se não existe
            if fn_hash not in all_footnotes:
                all_footnotes[fn_hash] = text
            
            # Adicionar à bridge
            bridge_data.append({
                'temp_row_id': idx,  # Temporário, será substituído por wage_id
                'footnote_hash': fn_hash,
                'column_reference': column_ref
            })

# Criar Dim_Footnotes
dim_footnotes = pd.DataFrame([
    {'footnote_hash': h, 'footnote_text': t} 
    for h, t in all_footnotes.items()
])
dim_footnotes['footnote_id'] = dim_footnotes.index + 1

# ============================================
# 6. CRIAR FACT_MINIMUMWAGE
# ============================================

fact_columns = ['state_id', 'category_id', 'year', 'base_wage_per_hour',
                'minimum_cash_wage', 'maximum_tip_credit', 'frequency', 'source_url']

fact_minimum_wage = df_unified[fact_columns].copy()
fact_minimum_wage['wage_id'] = fact_minimum_wage.index + 1
fact_minimum_wage['effective_date'] = None  # Preencher se tiver essa info

# Remover coluna notes da fato (agora está normalizada)
# Mas manter notas gerais não estruturadas se houver
fact_minimum_wage['notes'] = df_unified['notes'].apply(
    lambda x: x if pd.notna(x) and '[' not in str(x) else None
)

# Reordenar colunas
fact_minimum_wage = fact_minimum_wage[[
    'wage_id', 'state_id', 'category_id', 'year', 'effective_date',
    'base_wage_per_hour', 'minimum_cash_wage', 'maximum_tip_credit',
    'frequency', 'source_url', 'notes'
]]

# ============================================
# 7. CRIAR BRIDGE_WAGE_FOOTNOTE
# ============================================

bridge_df = pd.DataFrame(bridge_data)

# Substituir temp_row_id por wage_id
bridge_df['wage_id'] = bridge_df['temp_row_id'].map(
    dict(zip(df_unified.index, fact_minimum_wage['wage_id']))
)

# Adicionar footnote_id
bridge_df = bridge_df.merge(
    dim_footnotes[['footnote_hash', 'footnote_id']],
    on='footnote_hash',
    how='left'
)

# Limpar e criar bridge final
bridge_wage_footnote = bridge_df[['wage_id', 'footnote_id', 'column_reference']].drop_duplicates()

# ============================================
# 8. RESULTADOS FINAIS
# ============================================

print("=" * 60)
print("TABELAS CRIADAS:")
print("=" * 60)
print(f"✅ Fact_MinimumWage: {len(fact_minimum_wage)} registros")
print(f"✅ Dim_States: {len(dim_states)} registros")
print(f"✅ Dim_Categories: {len(dim_categories)} registros")
print(f"✅ Dim_Footnotes: {len(dim_footnotes)} registros")
print(f"✅ Bridge_Wage_Footnote: {len(bridge_wage_footnote)} registros")
print("=" * 60)

# Visualizar amostras
print("\n📊 Fact_MinimumWage (primeiras 5 linhas):")
display(fact_minimum_wage)

print("\n📊 Dim_Footnotes (primeiras 5 linhas):")
display(dim_footnotes)

print("\n📊 Bridge_Wage_Footnote (primeiras 5 linhas):")
display(bridge_wage_footnote)

/tmp/ipykernel_3433/26320032.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_unified = pd.concat([


TABELAS CRIADAS:
✅ Fact_MinimumWage: 5250 registros
✅ Dim_States: 60 registros
✅ Dim_Categories: 4 registros
✅ Dim_Footnotes: 241 registros
✅ Bridge_Wage_Footnote: 2273 registros

📊 Fact_MinimumWage (primeiras 5 linhas):


,wage_id,state_id,category_id,year,effective_date,base_wage_per_hour,minimum_cash_wage,maximum_tip_credit,frequency,source_url,notes
0,1,1,1,1968,None,1.15,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,"Or can be 1.60, this reflects which rates diff..."
1,2,2,1,1968,None,NaN,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,This state utilizes the federal minimum wage
2,3,3,1,1968,None,2.10,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,None
3,4,4,1,1968,None,18.72,NaN,NaN,3,https://www.dol.gov/agencies/whd/state/minimum...,"Or can be 26.40, this reflects which rates dif..."
4,5,5,1,1968,None,1.25,NaN,NaN,2,https://www.dol.gov/agencies/whd/state/minimum...,None
...,...,...,...,...,...,...,...,...,...,...,...
5245,5246,47,3,2024,None,NaN,NaN,9.87,1,https://www.dol.gov/agencies/whd/state/minimum...,None
5246,5247,47,4,2024,None,NaN,2.13,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,None
5247,5248,51,2,2024,None,7.25,NaN,NaN,1,https://www.dol.gov/agencies/whd/state/minimum...,None
5248,5249,51,3,2024,None,NaN,NaN,5.12,1,https://www.dol.gov/agencies/whd/state/minimum...,None



📊 Dim_Footnotes (primeiras 5 linhas):


,footnote_hash,footnote_text,footnote_id
0,8fe32f0077a4efbd,"Or can be 1.60, this reflects which rates diff...",1
1,dc2c1980e500b0b0,This state utilizes the federal minimum wage,2
2,85c5a92e0ddcbf71,"Or can be 26.40, this reflects which rates dif...",3
3,bc492112179d1b90,"Or can be 1.25, this reflects which rates diff...",4
4,4e46868784ca2194,"Or can be 75, this reflects which rates differ...",5
...,...,...,...
236,a64c0ba13f9f6f74,Tipped workers' minimum wage rates vary by reg...,237
237,8205a59a00a7a142,Ohio. For employees of employers with gross an...,238
238,246c23d8e69b5bf6,Oklahoma. For employers with fewer than 10 ful...,239
239,d450765abf11c2aa,Original value: 50% of the applicable minimum ...,240



📊 Bridge_Wage_Footnote (primeiras 5 linhas):


,wage_id,footnote_id,column_reference
0,1,1,general
1,2,2,general
2,4,3,general
3,7,4,general
4,10,2,general
...,...,...,...
2268,5243,226,jurisdiction
2269,5244,226,jurisdiction
2270,5248,229,jurisdiction
2271,5249,229,jurisdiction


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

# ==============================================================================
# FUNÇÕES DE LIMPEZA E TRANSFORMAÇÃO (DA SUA LÓGICA)
# ==============================================================================
# (Funções mantidas: add_leading_zero, process_multiple_rates_base, 
# process_tip_wages, convert_with_context, process_with_types)

def add_leading_zero(value):
    """Adiciona zero à esquerda se começar com '.' (ex: .75 -> 0.75)"""
    if isinstance(value, str):
        value = value.strip()
        if value.startswith('.'):
            return '0' + value
    return value

def process_multiple_rates_base(row):
    """
    Processa a coluna 'minimal_wage' (do scrape base) para:
    1. Extrair frequência (/day, /wk)
    2. Lidar com múltiplos valores (ex: 5.15 & 6.00)
    3. Identificar valores nulos para linkar com a nota federal
    """
    wage_str = row['minimal_wage_raw']
    new_notes = []
    
    # 1. Valor Nulo ou Faltante
    if pd.isna(wage_str) or wage_str in ['...', 'NA', '']:
        row['minimal_wage'] = None
        row['is_federal_rule'] = True # Flag para linkar footnote
        return row

    # 2. Detectar Frequência
    if '/day' in wage_str:
        row['frequency'] = 2 # 2 = Diário
        wage_str = wage_str.replace('/day', '').strip()
    elif '/wk' in wage_str:
        row['frequency'] = 3 # 3 = Semanal
        wage_str = wage_str.replace('/wk', '').strip()
    else:
        row['frequency'] = 1 # 1 = Por Hora (Default)

    # 3. Detectar múltiplos valores (ex: "$5.15 & 6.00" ou "5.25 - 6.15")
    pattern = r'\$?\d+\.?\d*'
    matches = re.findall(pattern, wage_str)
    
    valid_matches = [add_leading_zero(m.replace('$', '')) for m in matches if re.search(r'\d', m)]
    
    if len(valid_matches) >= 2:
        row['minimal_wage'] = valid_matches[0] # Pega o primeiro valor
        other_values = ", ".join(valid_matches[1:])
        note_text = f"[base_wage_per_hour] Taxas alternativas: {other_values}. Usando a primeira taxa."
        new_notes.append(note_text)
    elif len(valid_matches) == 1:
        row['minimal_wage'] = valid_matches[0]
    else:
        # Se não encontrar padrão numérico, é texto (ex: "None")
        row['minimal_wage'] = None
        if wage_str != "None": # Evitar notas desnecessárias
             new_notes.append(f"[base_wage_per_hour] Valor original: {wage_str}")

    # Adicionar notas, se houver
    if new_notes:
        if pd.notna(row['notes']):
            row['notes'] = f"{row['notes']} ; {' ; '.join(new_notes)}"
        else:
            row['notes'] = ' ; '.join(new_notes)
            
    return row

def process_tip_wages(row):
    """
    Processa valores de salário 'tipped', movendo texto para 'notes'
    (Sua função 'process_tip_wages' adaptada)
    """
    
    def is_monetary_value(value):
        if pd.isna(value) or not isinstance(value, str): return False
        clean = value.strip().replace('$', '')
        return bool(re.match(r'^\d+\.?\d*$', clean))
    
    def is_percentage(value):
        if pd.isna(value) or not isinstance(value, str): return False
        return '%' in value

    def move_text_to_notes(column_name, value, current_notes):
        """Move texto descritivo para notes"""
        if pd.isna(value) or not isinstance(value, str) or value.strip() == '':
            return None, current_notes
        
        value = value.strip()
        
        # Se não é valor monetário nem porcentagem, é texto descritivo
        if not is_monetary_value(value) and not is_percentage(value):
            note_text = f"[{column_name}] {value}"
            
            if pd.notna(current_notes):
                current_notes = f"{current_notes} ; {note_text}"
            else:
                current_notes = note_text
            
            return None, current_notes  # Limpar o valor original
        
        return value, current_notes

    current_notes = row.get('notes')
    
    # Processar cada coluna de valor
    for col in ['base_wage_per_hour', 'maximum_tip_credit', 'minimum_cash_wage']:
        if col in row:
            value, current_notes = move_text_to_notes(col, row[col], current_notes)
            row[col] = value
            
    row['notes'] = current_notes
    return row

def convert_with_context(value, column_name, row):
    """
    Converte valor para float e adiciona a coluna '_type' e notas.
    (Sua função 'convert_with_context' adaptada)
    """
    if pd.isna(value):
        return None, None, row
    
    if not isinstance(value, str):
        val = float(value) if isinstance(value, (int, float)) else None
        return val, 'exact', row
    
    original = value.strip()
    value = original.replace('$', '').strip()
    
    if value.lower() in ['not specified', 'missing value', '']:
        return None, None, row
    
    # Nota para adicionar ao campo 'notes'
    def add_note(row, note_text):
        if pd.notna(row.get('notes')):
            row['notes'] = f"{row['notes']} ; {note_text}"
        else:
            row['notes'] = note_text
        return row

    # Porcentagem
    if '%' in value:
        match = re.search(r'(\d+\.?\d*)\s*%', value)
        if match:
            note = f"[{column_name}] Valor original em %: {original}"
            row = add_note(row, note)
            return float(match.group(1)), 'percentage', row
    
    # Range (ex: "up to 5.00")
    range_patterns = {
        'up to': r'up to\s+(\d+\.?\d*)',
        'more than': r'more than\s+(\d+\.?\d*)',
        'at least': r'at least\s+(\d+\.?\d*)'
    }
    
    for range_type, pattern in range_patterns.items():
        match = re.search(pattern, value, re.IGNORECASE)
        if match:
            note = f"[{column_name}] {range_type.capitalize()} {match.group(1)} (Original: {original})"
            row = add_note(row, note)
            return float(match.group(1)), 'range', row
    
    # Exato
    try:
        # Tenta extrair o primeiro valor numérico
        match = re.search(r'(\d+\.?\d*)', value)
        if match:
            return float(match.group(1)), 'exact', row
        else:
            return None, None, row
    except (ValueError, TypeError):
        return None, None, row

def process_with_types(row):
    """Aplica a função 'convert_with_context' em todas as colunas de valor."""
    for col in ['base_wage_per_hour', 'maximum_tip_credit', 'minimum_cash_wage']:
        if col in row and pd.notna(row[col]):
            value, value_type, row = convert_with_context(row[col], col, row)
            row[col] = value
            row[f'{col}_type'] = value_type
    return row

# ==============================================================================
# FUNÇÃO 1: SCRAPE DO SALÁRIO MÍNIMO BÁSICO (HISTÓRICO)
# ==============================================================================
def scrape_base_wages():
    print("Iniciando scrape do salário mínimo básico (histórico)...")
    try:
        url = "https://www.dol.gov/agencies/whd/state/minimum-wage/history"
        page = requests.get(url, timeout=10)
        page.raise_for_status()
    except requests.RequestException as e:
        print(f"❌ Falha ao acessar a página de histórico: {e}")
        return pd.DataFrame(), {}

    soup = BeautifulSoup(page.content, 'html.parser')

    # 1. Extrair Footnotes
    footnotes_dict = {}
    content_div = soup.find('div', id='content')
    if content_div:
        for p in content_div.find_all('p'):
            text = p.get_text(strip=True)
            match = re.match(r'^([\[\(].*?[\]\)])', text)
            if match:
                ref_key = match.group(1)
                footnote_text = ' '.join(text.replace(ref_key, '').split()).replace('- ', '')
                unique_key = f"base_{ref_key}"
                footnotes_dict[unique_key] = footnote_text

    print(f"  > Encontrados {len(footnotes_dict)} footnotes de salário básico.")

    # 2. Extrair Tabela de Salários
    tabelas = soup.find_all('table')
    df_wage_list = []
    for tabela in tabelas:
        linhas = tabela.find_all('tr')
        if not linhas: continue
        
        cabecalho = linhas[0]
        anos = []
        year_ref_map = {} # Mapeia ano (ex: "2006") para ref_key (ex: "base_[a]")
        
        for th in cabecalho.find_all('th')[1:]:
            year_text = th.get_text(strip=True)
            year_digits = re.sub(r'\D', '', year_text)
            
            if year_digits.isdigit():
                anos.append(year_digits)
                # Procura por footnotes no header do ano
                fn_match = re.search(r'([\[\(].*?[\]\)])', year_text)
                if fn_match:
                    ref_key = f"base_{fn_match.group(1)}"
                    if ref_key in footnotes_dict:
                        year_ref_map[year_digits] = ref_key

        estados_data = []
        for estado_row in linhas[1:]:
            cells = [td.get_text(strip=True) for td in estado_row.find_all('td')]
            if cells:
                estados_data.append(cells)
        
        if estados_data and len(anos) == len(estados_data[0]) - 1:
            df = pd.DataFrame(estados_data, columns=['state'] + anos)
            df_wage_list.append(df)
        
    if not df_wage_list:
        print("❌ Falha ao processar dados das tabelas de histórico.")
        return pd.DataFrame(), footnotes_dict

    df = pd.concat(df_wage_list, ignore_index=True)
    df = df.drop_duplicates(subset=['state']).reset_index(drop=True)

    # 3. Melt
    df_melted = df.melt(id_vars=['state'], var_name='year', value_name='minimal_wage_raw')
    df_melted['year'] = pd.to_numeric(df_melted['year'])
    
    # 4. Extrair Footnotes (do valor e do header do ano)
    def get_footnotes(value, year):
        refs = []
        # Adiciona footnote do header do ano, se existir
        if str(year) in year_ref_map:
            refs.append(year_ref_map[str(year)])
        # Adiciona footnotes do valor da célula
        if isinstance(value, str):
            found_refs = re.findall(r'([\[\(].*?[\]\)])', value)
            for ref in found_refs:
                unique_key = f"base_{ref}"
                if unique_key in footnotes_dict:
                    refs.append(unique_key)
        return list(set(refs)) # Lista de ref_keys únicas

    df_melted['footnote_refs'] = df_melted.apply(lambda row: get_footnotes(row['minimal_wage_raw'], row['year']), axis=1)

    # 5. Limpar valor (remover footnotes)
    df_melted['minimal_wage_raw'] = df_melted['minimal_wage_raw'].astype(str).str.replace(r'[\[\(].*?[\]\)]', '', regex=True).str.strip()
    
    # 6. Definir Categoria e colunas do Schema
    df_melted['category_name'] = "Standard" # Categoria "Standard" (ID 1)
    df_melted['source_url'] = url
    df_melted['notes'] = pd.NA
    df_melted['is_federal_rule'] = False # Flag para regra de NULL
    
    df_melted = df_melted.rename(columns={'state': 'state_name'})
    
    print("✅ Scrape do salário mínimo básico concluído.")
    return df_melted, footnotes_dict


# ==============================================================================
# FUNÇÃO 2: SCRAPE DO SALÁRIO "TIPPED" (COM GORJETA)
# ==============================================================================
def scrape_tipped_wages(start_year=2003, end_year=2024):
    """
    Extrai dados anuais de salário "tipped" e seus footnotes.
    Retorna um DataFrame de dados brutos e um dicionário de footnotes.
    """
    print(f"Iniciando scrape do salário 'tipped' (Anos: {start_year}-{end_year})...")
    
    all_rows_data = []
    master_footnotes = {}

    for year in range(start_year, end_year + 1):
        url = f'https://www.dol.gov/agencies/whd/state/minimum-wage/tipped/{year}'
        try:
            tip_test = requests.get(url, timeout=10)
            if tip_test.status_code != 200:
                # print(f"  ⚠️ Falha ao obter dados de {year} (status {tip_test.status_code})")
                continue
        except requests.RequestException as e:
            # print(f"  ⚠️ Erro de conexão para o ano {year}: {e}")
            continue

        tip_soup = BeautifulSoup(tip_test.content, 'html.parser')

        # 1. EXTRAIR FOOTNOTES (com chave única por ano)
        year_footnotes = {}
        for a_tag in tip_soup.find_all('a', attrs={'name': lambda x: x and x.startswith('foot')}):
            name = a_tag.get('name') # ex: 'foot1'
            parent_p = a_tag.find_parent('p')
            if parent_p:
                footnote_num = a_tag.get_text(strip=True)
                texto_completo = ' '.join(parent_p.get_text().split())
                texto_nota = texto_completo.replace(footnote_num, '', 1).strip()
                unique_key = f"tip_{year}_{name}"
                year_footnotes[name] = unique_key
                master_footnotes[unique_key] = texto_nota

        # 2. PROCESSAR TABELA
        tip_table = tip_soup.find('table')
        if not tip_table: continue

        tip_linhas = tip_table.find_all('tr')[1:]
        ultima_jurisdiction = None
        
        for tr in tip_linhas:
            tds = tr.find_all('td')
            if not tds or tds[0].get('colspan'): continue

            row_data = {'year': year, 'source_url': url, 'notes': pd.NA}
            row_footnote_refs = []

            # Processar Jurisdição (Coluna 0)
            td_jurisdiction = tds[0]
            if td_jurisdiction.find('strong'):
                ultima_jurisdiction = td_jurisdiction.find('strong').get_text(strip=True)
            row_data['state_name'] = ultima_jurisdiction
            
            # Extrair refs da célula de jurisdição
            for link in td_jurisdiction.find_all('a', href=True):
                match = re.search(r'#(foot\d+)', link.get('href'))
                if match and match.group(1) in year_footnotes:
                    row_footnote_refs.append(year_footnotes[match.group(1)])

            # Extrair VALOR BRUTO (string) e REFS
            def extract_raw_cell(td_element):
                if not td_element: return None, []
                refs = []
                for link in td_element.find_all('a', href=True):
                    match = re.search(r'#(foot\d+)', link.get('href'))
                    if match and match.group(1) in year_footnotes:
                        refs.append(year_footnotes[match.group(1)])
                # Pega o texto bruto, incluindo "$", "%", "See note", etc.
                raw_text = td_element.get_text(strip=True)
                return raw_text if raw_text else None, refs

            # Coluna 1: combinedrate (base_wage_per_hour)
            if len(tds) > 1:
                val, refs = extract_raw_cell(tds[1])
                row_data['base_wage_per_hour'] = val
                row_footnote_refs.extend(refs)

            # Coluna 2: tipcredit (maximum_tip_credit)
            if len(tds) > 2:
                val, refs = extract_raw_cell(tds[2])
                row_data['maximum_tip_credit'] = val
                row_footnote_refs.extend(refs)

            # Coluna 3: cashwage (minimum_cash_wage)
            if len(tds) > 3:
                val, refs = extract_raw_cell(tds[3])
                row_data['minimum_cash_wage'] = val
                row_footnote_refs.extend(refs)

            # ----- INÍCIO DA LÓGICA CORRIGIDA -----
            # Categoria é sempre "Tipped"
            row_data['category_name'] = "Tipped"
            
            # Coluna 4 (Definition) -> Salva em 'sector_definition'
            if len(tds) > 4:
                sector_text = tds[4].get_text(strip=True)
                row_data['sector_definition'] = sector_text if sector_text else None
            else:
                row_data['sector_definition'] = None
            # ----- FIM DA LÓGICA CORRIGIDA -----


            row_data['footnote_refs'] = list(set(row_footnote_refs))
            all_rows_data.append(row_data)
        
    df_tips = pd.DataFrame(all_rows_data)
    print(f"✅ Scrape do salário 'tipped' concluído. Total de {len(df_tips)} registros.")
    return df_tips, master_footnotes


# ==============================================================================
# FUNÇÃO 3: ORQUESTRADOR PRINCIPAL E CONSTRUÇÃO DO SCHEMA
# ==============================================================================
def build_schema_tables(df_base_raw, base_footnotes, df_tipped_raw, tip_footnotes):
    """
    Constrói as 5 tabelas do Star Schema a partir dos dados brutos,
    aplicando a lógica de limpeza do usuário.
    """
    print("\nIniciando construção do Star Schema...")

    # ----- 1. Padronização de Jurisdições (Dim_States) -----
    state_name_map = {
        "Dist. of Columbia": "District of Columbia",
        "N. Mariana Islands": "Northern Mariana Islands",
        "Federal (FLSA)": "Federal" # Padronizar 'Federal'
    }
    df_base_raw['state_name'] = df_base_raw['state_name'].replace(state_name_map).str.strip()
    df_tipped_raw['state_name'] = df_tipped_raw['state_name'].replace(state_name_map).str.strip()

    all_states = pd.concat([df_base_raw['state_name'], df_tipped_raw['state_name']]).unique()
    all_states = sorted([s for s in all_states if pd.notna(s)])
    
    dim_states = pd.DataFrame({'state_name': all_states})
    dim_states.insert(0, 'state_id', range(1, len(dim_states) + 1))
    
    territories = ["American Samoa", "Guam", "Northern Mariana Islands", "Puerto Rico", "U.S. Virgin Islands", "District of Columbia"]
    dim_states['is_territory'] = dim_states['state_name'].isin(territories)
    print("  > Dim_States criada.")

    # ----- 2. Dim_Categories (LÓGICA CORRIGIDA) -----
    # Hardcoded conforme sua regra de negócio (1=Standard, 2=Tipped)
    categories_data = [
        {'category_id': 1, 'category_name': 'Standard'},
        {'category_id': 2, 'category_name': 'Tipped'}
    ]
    dim_categories = pd.DataFrame(categories_data)
    print("  > Dim_Categories criada (Standard=1, Tipped=2).")
    
    # Ajustar os DFs brutos para garantir que os nomes correspondam
    df_base_raw['category_name'] = "Standard"
    df_tipped_raw['category_name'] = "Tipped"

    # ----- 3. Dim_Footnotes (Implementando sua regra de NULL) -----
    NULL_WAGE_NOTE_TEXT = "Este estado não possui uma lei de salário mínimo; portanto, aplica-se a lei federal (FLSA)."
    NULL_WAGE_REF_KEY = "ref_key_federal_rule"
    
    master_footnote_dict = {**base_footnotes, **tip_footnotes}
    if NULL_WAGE_REF_KEY not in master_footnote_dict:
        master_footnote_dict[NULL_WAGE_REF_KEY] = NULL_WAGE_NOTE_TEXT

    df_fn = pd.DataFrame(master_footnote_dict.items(), columns=['ref_key', 'footnote_text'])
    df_fn = df_fn.drop_duplicates(subset=['footnote_text'])
    df_fn.insert(0, 'footnote_id', range(1, len(df_fn) + 1))
    dim_footnotes = df_fn
    
    # Buscar o ID da nota de rodapé federal que acabamos de criar
    federal_note_ref = dim_footnotes[dim_footnotes['ref_key'] == NULL_WAGE_REF_KEY]['ref_key'].iloc[0]
    
    print(f"  > Dim_Footnotes criada com {len(dim_footnotes)} notas (incluindo regra federal).")

    # ----- 4. Transformação e Criação da Fact_MinimumWage -----
    
    # 4.1 Processar Dados BASE (Standard)
    print("  > Processando dados 'Standard' (com sua lógica)...")
    df_base_processed = df_base_raw.apply(process_multiple_rates_base, axis=1)
    
    # Linkar o footnote federal (Regra de NULL)
    def link_federal_note(row):
        if row['is_federal_rule']:
            # Adiciona a ref_key à lista de refs para a bridge
            row['footnote_refs'].append(federal_note_ref)
        return row
    df_base_processed = df_base_processed.apply(link_federal_note, axis=1)
    df_base_processed = df_base_processed.drop(columns=['is_federal_rule', 'minimal_wage_raw'])
    
    # 4.2 Processar Dados TIPPED
    print("  > Processando dados 'Tipped' (com sua lógica)...")
    df_tipped_processed = df_tipped_raw.apply(process_tip_wages, axis=1)
    # Adicionar coluna 'frequency' padrão para tipped
    df_tipped_processed['frequency'] = 1 # Tipped é sempre por hora
    
    # 4.3 Unir os dataframes
    df_fact_raw = pd.concat([df_base_processed, df_tipped_processed], ignore_index=True, sort=False)
    
    # 4.4 Aplicar CONVERSÃO DE TIPO (Sua função 'process_with_types')
    print("  > Aplicando conversão de tipos (exact, percentage, range)...")
    df_fact_raw = df_fact_raw.apply(process_with_types, axis=1)

    # 4.5 Montar Tabela Fato Final
    # Substituir Nomes por IDs (FKs)
    df_fact_raw = df_fact_raw.merge(dim_states, on='state_name', how='left')
    df_fact_raw = df_fact_raw.merge(dim_categories, on='category_name', how='left')
    
    # Adicionar PK (wage_id)
    df_fact_raw.insert(0, 'wage_id', range(1, len(df_fact_raw) + 1))
    
    # Selecionar e ordenar colunas para a tabela Fato
    fact_columns = [
        'wage_id', 'state_id', 'category_id', 'year', 
        'base_wage_per_hour', 'minimum_cash_wage', 'maximum_tip_credit',
        'frequency', 'source_url', 'notes',
        # Colunas de tipo que você criou
        'base_wage_per_hour_type', 'minimum_cash_wage_type', 'maximum_tip_credit_type',
        # ----- NOVA COLUNA -----
        'sector_definition'
    ]
    # Garantir que todas as colunas existam (preenchendo com NA se faltarem)
    for col in fact_columns:
        if col not in df_fact_raw.columns:
            df_fact_raw[col] = pd.NA
            
    fact_minimum_wage = df_fact_raw[fact_columns].copy()
    fact_minimum_wage['effective_date'] = pd.to_datetime(fact_minimum_wage['year'].astype(str) + '-01-01')
    
    # Filtrar registros sem state_id ou category_id (dados "sujos" que não mapearam)
    fact_minimum_wage = fact_minimum_wage.dropna(subset=['state_id', 'category_id'])
    # Converter IDs para inteiros
    fact_minimum_wage['state_id'] = fact_minimum_wage['state_id'].astype(int)
    fact_minimum_wage['category_id'] = fact_minimum_wage['category_id'].astype(int)
    
    print(f"  > Fact_MinimumWage criada com {len(fact_minimum_wage)} registros.")
    
    # ----- 5. Bridge_Wage_Footnote -----
    # Usar a tabela de fatos (antes de dropar colunas) para pegar wage_id e footnote_refs
    df_bridge_raw = df_fact_raw[['wage_id', 'footnote_refs']].dropna(subset=['footnote_refs'])
    
    # Explodir a lista de refs
    df_bridge_exploded = df_bridge_raw.explode('footnote_refs')
    df_bridge_exploded = df_bridge_exploded.rename(columns={'footnote_refs': 'ref_key'})
    
    # Fazer merge com dim_footnotes para pegar o footnote_id
    df_bridge = df_bridge_exploded.merge(dim_footnotes[['ref_key', 'footnote_id']], on='ref_key', how='left')
    
    # Selecionar colunas finais da ponte
    bridge_wage_footnote = df_bridge[['wage_id', 'footnote_id']].copy()
    bridge_wage_footnote = bridge_wage_footnote.drop_duplicates().dropna()
    bridge_wage_footnote['wage_id'] = bridge_wage_footnote['wage_id'].astype(int)
    bridge_wage_footnote['footnote_id'] = bridge_wage_footnote['footnote_id'].astype(int)
    
    print(f"  > Bridge_Wage_Footnote criada com {len(bridge_wage_footnote)} links.")

    print("\n✅ Processamento do Schema concluído.")
    
    return dim_states, dim_categories, dim_footnotes, fact_minimum_wage, bridge_wage_footnote

# ==============================================================================
# EXECUÇÃO
# ==============================================================================
if __name__ == "__main__":
    # 1. Rodar os scrapers
    df_base_raw, base_notes_dict = scrape_base_wages()
    df_tipped_raw, tipped_notes_dict = scrape_tipped_wages(start_year=2003, end_year=2024)

    # 2. Construir as tabelas
    dim_states, dim_categories, dim_footnotes, fact_minimum_wage, bridge_wage_footnote = build_schema_tables(
        df_base_raw, base_notes_dict,
        df_tipped_raw, tipped_notes_dict
    )

    # 3. Mostrar resultados
    print("\n--- AMOSTRA DOS DADOS GERADOS ---")

    print("\n## Dim_States (Amostra)")
    print(dim_states.sample(min(5, len(dim_states))).to_markdown(index=False))

    print("\n## Dim_Categories (LÓGICA CORRIGIDA)")
    print(dim_categories.to_markdown(index=False))

    print("\n## Fact_MinimumWage (Amostra 'Tipped' com a nova coluna 'sector_definition')")
    # Mostrar registros 'Tipped' que agora têm a coluna 'sector_definition' preenchida
    print(fact_minimum_wage[
        (fact_minimum_wage['category_id'] == 2) & 
        (fact_minimum_wage['sector_definition'].notna())
    ].sample(min(5, len(fact_minimum_wage))).to_markdown(index=False))
    
    print("\n## Fact_MinimumWage (Amostra 'Standard')")
    print(fact_minimum_wage[fact_minimum_wage['category_id'] == 1].sample(min(5, len(fact_minimum_wage))).to_markdown(index=False))

    print("\n## Bridge_Wage_Footnote (Amostra)")
    print(bridge_wage_footnote.sample(min(5, len(bridge_wage_footnote))).to_markdown(index=False))

print("Dim State")
display(dim_states)
print("Dim Categories")
display(dim_categories)
print("Dim Footnotes")
display(dim_footnotes)
print("Fact Minimum Wage")
display(fact_minimum_wage)
print("Bridge Wage Footnote")
display(bridge_wage_footnote)


Iniciando scrape do salário mínimo básico (histórico)...
  > Encontrados 10 footnotes de salário básico.
✅ Scrape do salário mínimo básico concluído.
Iniciando scrape do salário 'tipped' (Anos: 2003-2024)...
✅ Scrape do salário 'tipped' concluído. Total de 1548 registros.

Iniciando construção do Star Schema...
  > Dim_States criada.
  > Dim_Categories criada (Standard=1, Tipped=2).
  > Dim_Footnotes criada com 75 notas (incluindo regra federal).
  > Processando dados 'Standard' (com sua lógica)...
  > Processando dados 'Tipped' (com sua lógica)...
  > Aplicando conversão de tipos (exact, percentage, range)...
  > Fact_MinimumWage criada com 3693 registros.
  > Bridge_Wage_Footnote criada com 200 links.

✅ Processamento do Schema concluído.

--- AMOSTRA DOS DADOS GERADOS ---

## Dim_States (Amostra)
|   state_id | state_name            | is_territory   |
|-----------:|:----------------------|:---------------|
|         60 | New                   | False          |
|            |       

,state_id,state_name,is_territory
0,1,Alabama,False
1,2,Alabama9,False
2,3,Alaska,False
3,4,American Samoa,True
4,5,Arizona,False
...,...,...,...
138,139,Wisconsin,False
139,140,Wisconsin11,False
140,141,Wisconsin12,False
141,142,Wisconsin8,False


Dim Categories


,category_id,category_name
0,1,Standard
1,2,Tipped


Dim Footnotes


,footnote_id,ref_key,footnote_text
0,1,base_(a),under the Federal Fair Labor Standards Act (FL...
1,2,base_(b),"For the years indicated, the laws in Arizona, ..."
2,3,base_[c],Rates applicable to employers of four or more.
3,4,base_(d),Rates applicable to employers of six or more. ...
4,5,base_(e),Rates applicable to employers of two or more.
...,...,...,...
242,71,tip_2024_foot10,"Nevada. Effective July 1, 2024, there is a uni..."
245,72,tip_2024_foot14,Connecticut. The Connecticut minimum wage is a...
246,73,tip_2024_foot15,District of Columbia. The minimum cash wage fo...
249,74,tip_2024_foot18,Puerto Rico. The minimum wage increased to $10...


Fact Minimum Wage


,wage_id,state_id,category_id,year,base_wage_per_hour,minimum_cash_wage,maximum_tip_credit,frequency,source_url,notes,base_wage_per_hour_type,minimum_cash_wage_type,maximum_tip_credit_type,sector_definition,effective_date
0,1,21,1,1968,NaN,NaN,NaN,1.0,https://www.dol.gov/agencies/whd/state/minimum...,[base_wage_per_hour] Taxas alternativas: 1.60....,NaN,NaN,NaN,NaN,1968-01-01
1,2,1,1,1968,NaN,NaN,NaN,NaN,https://www.dol.gov/agencies/whd/state/minimum...,<NA>,NaN,NaN,NaN,NaN,1968-01-01
2,3,3,1,1968,NaN,NaN,NaN,1.0,https://www.dol.gov/agencies/whd/state/minimum...,<NA>,NaN,NaN,NaN,NaN,1968-01-01
3,4,5,1,1968,NaN,NaN,NaN,3.0,https://www.dol.gov/agencies/whd/state/minimum...,[base_wage_per_hour] Taxas alternativas: 26.40...,NaN,NaN,NaN,NaN,1968-01-01
4,5,6,1,1968,NaN,NaN,NaN,2.0,https://www.dol.gov/agencies/whd/state/minimum...,<NA>,NaN,NaN,NaN,NaN,1968-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688,3689,120,2,2024,7.25,2.13,5.12,1.0,https://www.dol.gov/agencies/whd/state/minimum...,<NA>,exact,exact,exact,More than $30,2024-01-01
3689,3690,122,2,2024,7.25,2.13,5.12,1.0,https://www.dol.gov/agencies/whd/state/minimum...,<NA>,exact,exact,exact,More than $20,2024-01-01
3690,3691,124,2,2024,7.25,2.13,5.12,1.0,https://www.dol.gov/agencies/whd/state/minimum...,<NA>,exact,exact,exact,More than $30,2024-01-01
3691,3692,132,2,2024,12.00,2.13,9.87,1.0,https://www.dol.gov/agencies/whd/state/minimum...,<NA>,exact,exact,exact,More than $30,2024-01-01


Bridge Wage Footnote


,wage_id,footnote_id
1,2,75
3,4,2
4,5,2
5,6,2
6,7,2
...,...,...
3660,3651,73
3672,3663,68
3676,3667,69
3677,3668,70


In [ ]:
display(dim_footnotes)

,footnote_id,ref_key,footnote_text
0,1,base_(a),under the Federal Fair Labor Standards Act (FL...
1,2,base_(b),"For the years indicated, the laws in Arizona, ..."
2,3,base_[c],Rates applicable to employers of four or more.
3,4,base_(d),Rates applicable to employers of six or more. ...
4,5,base_(e),Rates applicable to employers of two or more.
...,...,...,...
242,71,tip_2024_foot10,"Nevada. Effective July 1, 2024, there is a uni..."
245,72,tip_2024_foot14,Connecticut. The Connecticut minimum wage is a...
246,73,tip_2024_foot15,District of Columbia. The minimum cash wage fo...
249,74,tip_2024_foot18,Puerto Rico. The minimum wage increased to $10...


In [ ]:
fact_minimum_wage['category_id'].value_counts()

category_id
118    385
1      261
122    193
124    155
6      132
      ... 
116      1
38       1
125      1
48       1
55       1
Name: count, Length: 126, dtype: int64

In [ ]:
df_tips

,jurisdiction,combinedrate,tipcredit,cashwage,definition,notes,year,combinedrate_type,tipcredit_type,cashwage_type
0,FEDERAL,5.15,3.02,2.13,More than $30,:\n \n Fair \n Labor Standa...,2003,exact,exact,exact
1,Alaska,NaN,NaN,7.15,NaN,"[cashwage] Alaska . Beginning January 1, 2004,...",2003,None,None,exact
2,California,NaN,NaN,6.75,NaN,NaN,2003,None,None,exact
3,Guam,NaN,NaN,5.15,NaN,NaN,2003,None,None,exact
4,Minnesota,NaN,NaN,5.15,Large employer,[definition] Minnesota . A large employer is a...,2003,None,None,exact
...,...,...,...,...,...,...,...,...,...,...
1384,Tennessee,7.25,5.12,2.13,More than $30,The following states do not have state minimum...,2024,exact,exact,exact
1385,Texas,7.25,5.12,2.13,More than $20,"The minimum wage laws in Kansas, Oklahoma (see...",2024,exact,exact,exact
1386,Utah,7.25,5.12,2.13,More than $30,"The minimum wage laws in Kansas, Oklahoma (see...",2024,exact,exact,exact
1387,Virginia,12.00,9.87,2.13,More than $30,NaN,2024,exact,exact,exact
